In [6]:
import pywt
import pyedflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import irfft, rfft, rfftfreq
import scaleogram as scg 
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import io
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [35]:
#Загрузка данных с fp1 канала
data_all = pd.read_csv('bioradio_fp1_all.csv',index_col=0)
data_all

,0,1,2,3,4,5,6,7,8,9,...,458,459,460,461,462,463,464,465,466,467
0,-38.688981,-41.475837,-19.619684,-21.522597,-18.774514,-1.135920,10.201555,33.534525,51.164200,22.611176,...,3.984794,-4.389011,-9.433942,-1.605755e+01,-1.401515e+01,-4.526525e+00,-9.481633e+00,-1.216306e+01,7.491432e+00,0.568779
1,36.605499,66.519832,23.202471,1.770098,17.926819,19.756842,12.061603,15.717185,30.203148,42.551918,...,-4.437099,-13.119685,-28.996098,-3.281517e+01,-3.314160e+01,-5.294750e+01,-5.380674e+01,-3.405058e+01,-4.348002e+01,-33.340536
2,-22.398778,-33.974496,-15.158805,-11.568783,-15.718942,-16.050849,-17.673722,-21.144579,-29.537545,-30.998016,...,28.005902,42.007342,58.282047,2.615001e+01,-1.268454e+01,-1.996253e+00,3.422842e+00,-7.038508e+00,1.316829e+01,16.346295
3,-7.425061,-6.513222,-1.236387,11.251677,7.488835,5.066947,22.600745,22.659554,0.928947,-5.322132,...,-30.460142,8.545752,22.393095,1.938898e+01,1.194275e+01,-5.118158e+00,-1.327163e+01,-8.274760e+00,-1.119835e+01,-13.751303
4,-42.222584,-35.927321,4.388841,18.023850,13.380245,17.777252,10.560577,-7.043890,-9.743083,1.926370,...,-5.734123,-17.537391,-28.940559,-1.888614e+01,3.135940e+00,1.632085e+01,1.010532e+01,3.671677e-01,3.409411e+00,-9.789412
5,49.097745,69.797507,36.340735,23.747420,22.909190,12.230372,9.202059,9.775493,-6.765573,-37.745537,...,-49.990915,-22.957288,-4.749072,6.399243e-03,1.173495e+01,2.424256e+01,2.939861e+01,4.262520e+00,-4.231078e+01,-25.757333
6,-62.649821,-83.260508,-38.041246,9.106710,29.406771,46.486562,51.716045,47.594100,37.829456,14.539158,...,8.864967,10.349807,2.113127,-6.184574e+00,-5.022370e+00,-1.173137e+01,-2.571749e+01,-3.123685e+01,-1.927151e+01,-20.276325
7,-10.409269,-43.061751,-25.109637,-13.292710,-17.949294,-7.726711,0.237263,4.904802,15.707322,9.623993,...,20.390336,5.356536,9.687888,-1.164902e+00,-1.848254e+01,-9.370088e+00,-1.453460e+00,-1.013020e+01,7.981640e+00,26.399031
8,27.821180,65.039066,33.930561,29.742452,29.837870,6.035959,8.522180,33.388785,34.758997,4.560880,...,8.112555,-42.629915,-67.739546,-6.800762e+01,-7.560079e+01,-4.921782e+01,3.400768e+01,5.837014e+01,-2.461328e+01,-53.219540
9,144.165658,173.177857,68.639641,38.637563,39.212889,11.435904,1.269508,-12.345508,-32.154362,-33.131200,...,-31.104676,-21.102198,-41.963229,-4.984132e+01,-3.453624e+01,-5.151911e+01,-5.128214e+01,-5.023595e+01,-1.011706e+02,-38.693022


In [353]:
#Извлечение ритмов при помощи вейвлет преобразования
# gamma_rhythm = []
# beta_rhythm = []
# alpha_rhythm = []
# theta_rhythm = []
# delta_rhythm = []
# for i in range(len(data_all)):
#     coeff = pywt.swt(data_all.iloc[i], 'db4', level=4)
#     gamma_rhythm.append(coeff[3][1]) #Результат первого разложения 30-60Гц, Коэффициенты детализации 
#     beta_rhythm.append(coeff[2][1]) #Результат второго разложения 15-30Гц, Коэффициенты детализации 
#     alpha_rhythm.append(coeff[1][1]) #Результат третьего разложения 8-15Гц, Коэффициенты детализации 
#     theta_rhythm.append(coeff[0][1]) #Результат четвортого разложения 4-8Гц, Коэффициенты детализации 
#     delta_rhythm.append(coeff[0][0]) #Результат четвертого разложения 0-4Гц, Коэффициенты аппроксимации

In [36]:
# Извлечение ритмов при помощи преобразования Фурье 
# Разкомментировать Ctrl + /
gamma_rhythm = []
beta_rhythm = []
alpha_rhythm = []
theta_rhythm = []
delta_rhythm = []
for i in range(len(data_all)):
    y = rfft(data_all.iloc[i].to_numpy()) # мощность волн определённой частоты 
    x = rfftfreq(len(data_all.iloc[i].to_numpy()), 1 / 128) # список всех частот 

    alphaX = x.copy()
    alphaY = y.copy()
    betaX = x.copy()
    betaY = y.copy()
    gammaX = x.copy()
    gammaY = y.copy()
    thetaX = x.copy()
    thetaY = y.copy()
    deltaX = x.copy()
    deltaY = y.copy()


    for i in range(len(alphaX)):
        if (alphaX[i] < 8) or (alphaX[i] > 13):
            alphaY[i] = 0
        
    for i in range(len(betaX)):
        if (betaX[i] < 14) or (betaX[i] > 40):
            betaY[i] = 0
        
    for i in range(len(gammaX)):
        if gammaX[i] < 40:
            gammaY[i] = 0

    for i in range(len(thetaX)):
        if (thetaX[i] < 4) or (thetaX[i] > 8):
            thetaY[i] = 0
    
    for i in range(len(deltaX)):
        if deltaX[i] > 4:
            deltaY[i] = 0
    
    filteredAlpha = irfft(alphaY) # Восстановление графика ЭЭГ по частотам
    filteredBeta = irfft(betaY)
    filteredGamma = irfft(gammaY) 
    filteredTheta = irfft(thetaY)
    filteredDelta = irfft(deltaY)
    
    gamma_rhythm.append(filteredGamma)
    beta_rhythm.append(filteredBeta)
    alpha_rhythm.append(filteredAlpha)
    theta_rhythm.append(filteredTheta)
    delta_rhythm.append(filteredDelta)
    

In [13]:
# Загрузка модели alexnet
model = models.alexnet(weights='DEFAULT')
train_nodes, eval_nodes = get_graph_node_names(model)
create_feature_extractor(model, return_nodes=eval_nodes) # просмотр структуры

AlexNet(
  (features): Module(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Module(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): 

In [14]:
# Получение новой модели alexnet для извлечения признаков с классификатора №3
new_model = create_feature_extractor(model, return_nodes=['classifier.4'])

In [15]:
# Загрузка модели VGG16
model_vgg = models.vgg16(weights='DEFAULT')
train_nodes_vgg, eval_nodes_vgg = get_graph_node_names(model_vgg)
create_feature_extractor(model_vgg, return_nodes=eval_nodes_vgg) #Просмотр структуры

VGG(
  (features): Module(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ce

In [16]:
# Получение новой модели VGG16 для извлечения признаков с классификатора №2
new_model_vgg = create_feature_extractor(model_vgg, return_nodes=['classifier.3']) 

In [17]:
#Вспомогательная функция для преобразования изображения в размер 244x244
transform = transforms.Compose([
  transforms.Resize([224,224]),
  transforms.ToTensor()                              
])

In [49]:
#Массивы для полученных признаков из CNN
input_svm_beta_alex = []
input_svm_beta_vgg = []

In [50]:
coikw = {'alpha': 0.5, 'hatch': '/'}

#Занимает много памяти в процессе, я делала каждые 100 в цикле (0,100),(100,200)..

for i in range(0,19):
    #Преобразование ритма в изображение
    plot = scg.cws(beta_rhythm[i], figsize = (3, 3), coikw=coikw, cbar=None)
    #Очистка графика от осей, заголовков
    plot.invert_yaxis()
    plot.set_title(None)
    plot.set_xlabel(None)
    plot.set_ylabel(None)
    plot.axis('off')

    #Сохранение изображения
    img_buf = io.BytesIO()
    plot.get_figure().savefig(img_buf, format='png')
    plt.close('all')

    #Загрузка изображения
    im = Image.open(img_buf)
    im = im.convert('RGB')
    
    #Преобразование данных изображения в формат tensor [1,3,224,224]
    input_tensor = transform(im)
    input_batch = input_tensor.unsqueeze(0) 
    img_buf.close()

    #Извлечение признаков
    features = new_model(input_batch)
    featuresVGG = new_model_vgg(input_batch)
    
    #Сохранение признаков
    input_svm_beta_alex.append(features['classifier.4'][0].detach().numpy())
    input_svm_beta_vgg.append(featuresVGG['classifier.3'][0].detach().numpy())
    
    #Для отслеживания в процессе цикла
    if i % 50 == 0:
        print(i)

0


In [51]:
#Сохранение в файл полученных признаков
pd.DataFrame(input_svm_beta_alex).to_csv('bioradio_fp1_alexnet_beta_fc6_clf4.csv')
pd.DataFrame(input_svm_beta_vgg).to_csv('bioradio_fp1_vgg_beta_fc6_clf3.csv')
print(len(input_svm_beta_alex),len(input_svm_beta_vgg))

19 19


In [355]:
#Загрузка признаков 
fp1_beta_alexnet = pd.read_csv('fp1_alexnet_beta_fc6_clf4.csv',index_col=0)

In [354]:
labels = pd.read_csv('deap_all_labels.csv',index_col=0)

In [13]:
labels_bioradio = pd.read_csv('bioradio_raiting_all.csv',index_col=0)
labels_bioradio

,valence,arousal,dominance,liking,valence_model,arousal_model
0,7,7,6,4,1,1
1,5,7,3,2,1,1
2,7,3,6,3,1,0
3,2,2,2,1,0,0
4,5,7,8,2,1,1
5,7,3,3,1,1,0
6,8,3,5,3,1,0
7,7,2,3,4,1,0
8,8,7,5,4,1,1
9,5,1,4,1,1,0


In [295]:
data_bioradio = pd.read_csv('bioradio_fp1_alexnet_beta_fc6_clf4.csv',index_col=0)

In [326]:
#Деление на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(fp1_beta_alexnet, labels['valence'], test_size=0.25, random_state=985) #, stratify=labels['valence']

In [327]:
#Таблица результатов
results_table = pd.DataFrame()
results_table['criterion'] = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
results_table

,criterion
0,accuracy_score
1,f1_score
2,precision_score
3,recall_score


## Метод опорных векторов

In [328]:
clf = svm.SVC(kernel='rbf',C=25)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [329]:
results_table['svc'] =[accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                       precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc
0,accuracy_score,0.842105
1,f1_score,0.909091
2,precision_score,0.882353
3,recall_score,0.937500


In [61]:
# #метод опорных векторов с gridsearch
# parameters_svc = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'C':[0.0001, 100], 'gamma':('scale', 'auto')} #'kernel':('linear', 'rbf'),
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters_svc, scoring = 'accuracy')
# clf.fit(X_train, y_train)
# results = clf.predict(X_test)
# clf.best_params_

In [62]:
# results_table['svc_grid'] = [accuracy_score(y_test,results), f1_score(y_test,results), precision_score(y_test,results), recall_score(y_test,results)]
# results_table

In [63]:
# #Поиск random_state
# best = 0.52
# best_random= 0
# params = {}
# for i in range(40,80):
#     X_train, X_test, y_train, y_test = train_test_split(fp1_alpha_alexnet, labels['valence'], test_size=0.25, random_state=i)
#     clf = svm.SVC(kernel='rbf',C=3)
#     clf.fit(X_train, y_train)
#     results = clf.predict(X_test)
#     if accuracy_score(y_test,results) > best:
#         best = accuracy_score(y_test,results)
#         best_random = i
#     if i % 10 == 0:
#         print(i)

In [64]:
# best

## Дерево решений

In [330]:
tree = DecisionTreeClassifier(criterion='gini',
 max_depth=3,
 min_samples_split=15,
 min_samples_leaf=10)
tree.fit(X_train, y_train)
results = tree.predict(data_bioradio)

In [331]:
results_table['tree'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                       precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree
0,accuracy_score,0.842105,0.736842
1,f1_score,0.909091,0.848485
2,precision_score,0.882353,0.823529
3,recall_score,0.937500,0.875000


In [332]:
print(classification_report(labels_bioradio['valence_model'],results))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.82      0.88      0.85        16

    accuracy                           0.74        19
   macro avg       0.41      0.44      0.42        19
weighted avg       0.69      0.74      0.71        19



## Случайный лес

In [335]:
clf = RandomForestClassifier(criterion='gini',
 n_estimators=4,
 max_depth=2,
 min_samples_leaf= 15,
 min_samples_split = 10)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [336]:
results_table['forest'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                       precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest
0,accuracy_score,0.842105,0.736842,0.473684
1,f1_score,0.909091,0.848485,0.583333
2,precision_score,0.882353,0.823529,0.875000
3,recall_score,0.937500,0.875000,0.437500


## Наивный Байесовский классификатор

In [337]:
data_nb_x_train = MinMaxScaler().fit(X_train).transform(X_train)
data_nb_x_test = MinMaxScaler().fit(data_bioradio).transform(data_bioradio)
clf = MultinomialNB(alpha=1)
clf.fit(data_nb_x_train, y_train)
results = clf.predict(data_nb_x_test)

In [338]:
results_table['nb'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                       precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb
0,accuracy_score,0.842105,0.736842,0.473684,0.631579
1,f1_score,0.909091,0.848485,0.583333,0.758621
2,precision_score,0.882353,0.823529,0.875000,0.846154
3,recall_score,0.937500,0.875000,0.437500,0.687500


## Метод ближайших соседей

In [339]:
clf = KNeighborsClassifier(n_neighbors=100,weights='distance',algorithm='auto')
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [340]:
results_table['knn'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                       precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000


## Bagging SVM

In [341]:
svc = svm.SVC(kernel='rbf',C=35)
clf = BaggingClassifier(base_estimator=svc, n_estimators=35, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [342]:
results_table['bagging_svm'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                               precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500


## Bagging дерево решений

In [343]:
tree = DecisionTreeClassifier(criterion='gini',
 max_depth=3,
 min_samples_split=15,
 min_samples_leaf=10)
clf = BaggingClassifier(base_estimator=tree, n_estimators=35, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [344]:
results_table['bagging_tree'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                               precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm,bagging_tree
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105,0.789474
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091,0.882353
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353,0.833333
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500,0.937500


## Bagging случайный лес

In [345]:
forest = RandomForestClassifier(criterion='gini',
 n_estimators=4,
 max_depth=2,
 min_samples_leaf= 15,
 min_samples_split = 10)
clf = BaggingClassifier(base_estimator=forest, n_estimators=15, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [346]:
results_table['bagging_forest'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                               precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm,bagging_tree,bagging_forest
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105,0.789474,0.894737
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091,0.882353,0.941176
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353,0.833333,0.888889
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500,0.937500,1.000000


## Stacking

In [347]:
estimators = [
    ('rf', RandomForestClassifier(criterion='gini',
 n_estimators=4,
 max_depth=2,
 min_samples_leaf= 15,
 min_samples_split = 10)),
    ('dt', DecisionTreeClassifier(criterion='gini',
 max_depth=3,
 min_samples_split=15,
 min_samples_leaf=10)),
    ('svc', svm.SVC(kernel='rbf',C=5)),
    ('knn',KNeighborsClassifier(n_neighbors=50,weights='distance',algorithm='auto'))
    ]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()) 
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [348]:
results_table['stacking'] =[accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                           precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm,bagging_tree,bagging_forest,stacking
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105,0.789474,0.894737,0.842105
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091,0.882353,0.941176,0.914286
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353,0.833333,0.888889,0.842105
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500,0.937500,1.000000,1.000000


## Boosting дерево решений

In [349]:
tree = DecisionTreeClassifier(criterion='gini',
 max_depth=3,
 min_samples_split=15,
 min_samples_leaf=10)
clf = AdaBoostClassifier(base_estimator=tree,n_estimators=35, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [350]:
results_table['boost_tree'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                               precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm,bagging_tree,bagging_forest,stacking,boost_tree
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105,0.789474,0.894737,0.842105,0.736842
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091,0.882353,0.941176,0.914286,0.848485
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353,0.833333,0.888889,0.842105,0.823529
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500,0.937500,1.000000,1.000000,0.875000


## Boosting случайный лес

In [351]:
tree = RandomForestClassifier(criterion='entropy',
 n_estimators=30,
 max_depth=3,
 min_samples_leaf= 15,
 min_samples_split = 5)
clf = AdaBoostClassifier(base_estimator=tree,n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(data_bioradio)

In [352]:
results_table['boost_forest'] = [accuracy_score(labels_bioradio['valence_model'],results), f1_score(labels_bioradio['valence_model'],results), \
                                precision_score(labels_bioradio['valence_model'],results), recall_score(labels_bioradio['valence_model'],results)]
results_table

,criterion,svc,tree,forest,nb,knn,bagging_svm,bagging_tree,bagging_forest,stacking,boost_tree,boost_forest
0,accuracy_score,0.842105,0.736842,0.473684,0.631579,0.789474,0.842105,0.789474,0.894737,0.842105,0.736842,0.789474
1,f1_score,0.909091,0.848485,0.583333,0.758621,0.875000,0.909091,0.882353,0.941176,0.914286,0.848485,0.875000
2,precision_score,0.882353,0.823529,0.875000,0.846154,0.875000,0.882353,0.833333,0.888889,0.842105,0.823529,0.875000
3,recall_score,0.937500,0.875000,0.437500,0.687500,0.875000,0.937500,0.937500,1.000000,1.000000,0.875000,0.875000
